## Introduction

Here we explore the relationship between KOLs/Influencers and Brands using network analysis. The creation of the network will be based on LLM-extraction.

In [ ]:
# Install required packages
!pip install ollama tqdm -q

## Setting up Ollama server

In [ ]:
# Install Ollama
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
import os
import threading
import subprocess

def start_ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=start_ollama)
ollama_thread.start()

In [ ]:
# Download pre-trained model
! ollama pull qwen2.5

## Load the dataset

Data is from HuggingFace **vector-institute/newsmediabias-plus**, which contains news articles. The structure is as follow:
- unique_id: Unique identifier for each news item.
- headline: Headline of the news article.
- article_text: Full text content of the news article.
- new_categories: Categories assigned to the article.

In [ ]:
# Import libraries
import pandas as pd
import json
import ollama
from tqdm import tqdm
from google.colab import drive


In [ ]:
# Data is reformat and save to google drive for faster retrieval
# Load data from google drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/articles_data.json'
with open(file_path, 'r') as f:
  df = json.load(f)

## Text Classification

Purpose: Decide if an articles may contains information about KOLs, Influencers, Brand Marketing, Events, Campaigns, Social Media...

This faciliate the relationship extraction step to focus only on potential articles

In [ ]:
## Define function for classification model
system_prompt = '''
Your task is to classify the text into one of two categories based on the following criteria:

- Output "yes" if the text is about topics like Youtubers, Tiktokers, Food Reviewers, Beauty Bloggers, Social Media Influencers, Influencer Marketing Campaigns, E-commerce Campaigns, Social Media Marketing Campaigns, Marketing Events, Fashion Events, Product Launch Events, Virtual Influencer Events, and Social Media Events, particularly content discussing collaborations between influencers/KOLs and brands or content aimed at audience engagement, social media campaigns, or brand promotion.
- Output "no" if the text is about unrelated topics, such as historical events, political figures, wars, or general public figures not related to influencer marketing, brand promotions, or entertainment.


Provide only one word: "yes" or "no".
'''

def article_classification(article):
    prompt = f"""
    Decide if the text is about Youtubers, Tiktokers, Food Reviewers, Beauty Bloggers, Social Media Influencers, Influencer Marketing Campaigns, E-commerce Campaigns, Social Media Marketing Campaigns, Marketing Events, Fashion Events, Product Launch Events, Virtual Influencer Events, and Social Media Events, or similar topics that might include people collaborations with Brands.
    Title: {article['headline']}
    Text: {article['article_text']}.
    Provide only one word: 'yes' or 'no'.
    """
    response = ollama.chat(
        model='qwen2.5',
        messages=[
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': prompt},
        ],
        format='text',
        options={"temperature": 0.0}  # setting temperature to 0 for determinism
    )

    # Ensure the response is constrained to "yes" or "no"
    classification = response['message']['content'].strip().lower()

    return classification


In [ ]:
is_relevant_result = {}

# Function to call classification method on each article and save the result for later use
def is_relevant_check(article):
  try:
      extracted_data = article_classification(article)
      is_relevant_result[article['unique_id']] = extracted_data
  except json.JSONDecodeError as e:
      print(f"Error processing page '{article['unique_id']}': {str(e)}")

# Processing
for article in tqdm(df, desc="Processing articles"):
    text = article['new_categories'].lower()
    if 'politics' in text:
      if 'business' in text or 'entertainment' in text or 'technology' in text:
        is_relevant_check(article)
    else:
      is_relevant_check(article)


# Saving result to a file
with open('is_relevant_result.json', 'w') as f:
    json.dump(is_relevant_result, f)

## Relationship Extraction

In [ ]:
import json
import ollama
import networkx as nx
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

#### Defining the Extraction Schema
Each relationship will include:
- influencer_name (from)
- brand_name (to)
- relationship_type (type)
- brand_category (category)

In [ ]:
PROMPT_STRUCTURE = """
RULES:
1. ONLY extract relationships that are EXPLICITLY stated in the input text
2. DO NOT use any examples provided as actual relationships
3. DO NOT make up or infer relationships not directly mentioned
4. IGNORE hypothetical or future potential relationships
5. If no valid relationships are found, return empty edges array

OUTPUT FORMAT:
{
  "edges": [
    {
      "from": "influencer_name",
      "to": "brand_name",
      "type": "relationship_type",
      "category": "brand_category"
    }
  ]
}

relationship_type MUST be one of the following:
- "ambassador": Official brand representative with formal agreement
- "reviewer": Creates product reviews or promotional content
- "partner": Collaborates on specific campaigns
- "endorser": Informally promotes or recommends products
- "collaborator": Co-creates products or marketing initiatives

brand_category MUST be one of the following:
- "Fashion and Apparel": Clothing, footwear, accessories
- "Beauty and Cosmetics": Skincare, makeup, hair care
- "Technology": Electronics, software, gadgets
- "Automotive": Vehicles and related products
- "Food and Beverage": Food products and drinks
- "Health and Wellness": Fitness, supplements, healthcare
- "Home Goods and Appliances": Furniture, decor
- "Entertainment and Media": Streaming, gaming, content
- "Finance and FinTech": Banking, financial services
- "Travel and Hospitality": Tourism, accommodations

VALIDATION STEPS:
1. Does the text explicitly name both an influencer AND a brand? If not, skip.
2. Is their relationship clearly defined? If not, skip.
3. Can you determine the brand category with certainty? If not, skip.
4. Is this a current/past relationship (not future/hypothetical)? If not, skip.

EXAMPLE INPUT:
Headline: Beauty Influencer Emma Smith Reviews New MAC Collection
Text: Popular beauty YouTuber Emma Smith posted an in-depth review of MAC Cosmetics' latest collection on her channel yesterday.

EXAMPLE OUTPUT:
{
  "edges": [
    {
      "from": "Emma Smith",
      "to": "MAC Cosmetics",
      "type": "reviewer",
      "category": "Beauty and Cosmetics"
    }
  ]
}

Return ONLY the JSON output, no additional text or explanations.
"""

SYSTEM_PROMPT = """You are a precise relationship extraction system. Your task is to analyze the given text and extract ONLY explicitly mentioned relationships between influencers and brands.

INPUT FORMAT:
You will receive text containing a headline and article content.

""" + PROMPT_STRUCTURE

In [ ]:
# Extraction function
def extract_relationships(article):
    prompt = f"""
    Analyze the following text and extract any explicit influencer-brand relationships:

    HEADLINE: {article['headline']}
    CONTENT: {article['article_text']}

    """ + PROMPT_STRUCTURE

    response = ollama.chat(
        model='qwen2.5',
        messages=[{'role': 'system', 'content': SYSTEM_PROMPT},
                  {'role': 'user', 'content': prompt},],
        format='json',
        options={"temperature": 0.1}
    )

    return response['message']['content']

#### Extraction Process

In [ ]:
# Load classification result file
with open('is_relevant_result_name.json', 'r') as f:
    is_relevant_result = json.load(f)

In [ ]:
# Process articles
results_relationship = []
for article in tqdm(df, desc="Processing articles"):
    if is_relevant_result.get(article['unique_id']) == 'yes':
      try:
          extracted_data = json.loads(extract_relationships(article))
          results_relationship.append(extracted_data)
      except json.JSONDecodeError as e:
          print(f"Error processing article '{article['unique_id']}': {str(e)}")

In [ ]:
# Save all results
with open('kol_brand_relationships.json', 'w', encoding='utf-8') as f:
    json.dump(results_relationship, f, ensure_ascii=False, indent=2)

print(f"relationship results saved")

## Graph construction using Neo4J

In [ ]:
!pip install neo4j tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 3.8 MB/s eta 0:00:00


In [ ]:
from neo4j import GraphDatabase
import json
from google.colab import userdata
from tqdm import tqdm
from collections import defaultdict

In [ ]:
# Neo4j connection details
uri = "neo4j+s://af29f150.databases.neo4j.io"
username = "neo4j"              # Default username
password = userdata.get('AURA_NEOJ4_PASS')

# Create a Neo4j driver instance
driver = GraphDatabase.driver(uri, auth=(username, password))

In [ ]:
# Define a function to extract entities and relationships
def extract_entities_and_relationships(data):
  entities = defaultdict(set)  # To store unique entities by type
  relationships = []
  print('Process data and create nodes and relationships')
  for item in tqdm(data):
    try:
      if item["edges"]:  # Skip empty edges
          for edge in item["edges"]:
              if not all(key in edge and edge[key] for key in ["from", "to", "type", "category"]):
                print('Invalid edge format:', edge)
                continue
              # Create nodes and relationship
              entities['Influencer'].add(edge['from'])
              entities[edge['category'].replace(' and ', '_').replace(' ', '_')].add(edge['to'])
              relationships.append({
                  'from': edge['from'],
                  'to': edge['to'],
                  'type': edge['type']
              })
    except:
      print('No edges found')
  return entities, relationships

#### Building cypher queries

In [ ]:
def clear_database():
    clear_query = "MATCH (n) DETACH DELETE n"
    with driver.session() as session:
        session.run(clear_query)
        print("Database cleared.")

In [ ]:
def generate_node_queries(entities):
    cypher_queries = []

    # Function to escape single quotes in Cypher queries
    def escape_single_quotes(text):
        return text.replace("'", "\\'")

    # Create queries for nodes (entities)
    for entity_type, entity_names in entities.items():
        for entity_name in entity_names:
            name_escaped = escape_single_quotes(entity_name)
            query = f"CREATE (e:{entity_type} {{name: '{name_escaped}'}})"  # Use the entity type as the label
            cypher_queries.append(query)

    return cypher_queries

In [ ]:
def generate_relationship_queries(relationships):
    cypher_queries = []

    # Function to escape single quotes in Cypher queries
    def escape_single_quotes(text):
        return text.replace("'", "\\'")

    # Create queries for relationships
    for relationship in relationships:
        if isinstance(relationship, dict) and 'from' in relationship and 'to' in relationship and 'type' in relationship:
            from_escaped = escape_single_quotes(relationship['from'])
            to_escaped = escape_single_quotes(relationship['to'])
            query = f"""
            MATCH (a {{name: '{from_escaped}'}}),
                  (b {{name: '{to_escaped}'}})
            MERGE (a)-[:{relationship['type']}]->(b)
            """
            cypher_queries.append(query)
        else:
            print(f"Skipping unexpected relationship structure: {relationship}")

    return cypher_queries

In [ ]:
def run_cypher_queries(queries):
    with driver.session() as session:
        for query in tqdm(queries):
            session.run(query)
            print(f"Executed: {query[:100]}...")  # Track progress

#### Executing

In [ ]:
with open('update-relationship.json', 'r') as f:
    data = json.load(f)

entities, relationships = extract_entities_and_relationships(data)
node_queries = generate_node_queries(entities)
relationship_queries = generate_relationship_queries(relationships)

clear_database()
run_cypher_queries(node_queries)

run_cypher_queries(relationship_queries)

  0%|          | 1/1090 [00:00<06:30,  2.79it/s]

Executed: 
            MATCH (a {name: 'Rudy Kelly'}),
                  (b {name: 'CBC Indigenous'})
        ...


  0%|          | 3/1090 [00:00<04:01,  4.50it/s]

Executed: 
            MATCH (a {name: 'Phyllis Webstad'}),
                  (b {name: 'CBC Books'})
        ...
Executed: 
            MATCH (a {name: 'Julie Delporte'}),
                  (b {name: 'Drawn & Quarterly'})
 ...


  0%|          | 5/1090 [00:01<03:17,  5.49it/s]

...

Executed: 
            MATCH (a {name: 'Alex Fraser'}),
                  (b {name: 'British Red Cross'})
    ...
Executed: 
            MATCH (a {name: 'Enver Solomon'}),
                  (b {name: 'Refugee Council'})
    ...


100%|█████████▉| 1089/1090 [02:40<00:00,  7.32it/s]

Executed: 
            MATCH (a {name: 'Gail Porter'}),
                  (b {name: 'Fair for You'})
         ...
Executed: 
            MATCH (a {name: 'Guy Barker'}),
                  (b {name: 'Incognito'})
            M...


100%|██████████| 1090/1090 [02:41<00:00,  6.77it/s]

Executed: 
            MATCH (a {name: 'Kamasi Washington'}),
                  (b {name: 'Flying Lotus'})
   ...
